## 필요 패키지 설치

허깅페이스의 트랜스포머스 라이브러리를 설치합니다.

In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-hmvycnu4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-hmvycnu4
  Resolved https://github.com/huggingface/accelerate to commit 162a82164e9bdcc01a173cbee43b686437aaead8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.27.0.dev0-py3-none-any.whl size=273010 sha256=aa05a59a2afc16577f0baaaeb8e6bb4e9ed9f6d73a6b85d62243012c7b9fa13c
  Stored in directory: /tmp/pip-ephem-wheel-cache-q3a4547k/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


데이터 셋을 다운받는기 위해 다음 명령을 실행해서 허깅페이스 `Datasets` 라이브러리를 설치합니다.

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


그리고 T5 모델의 토크나이저가 `sentencepiece`를 사용하므로 다음을 실행해서 설치합니다.

In [4]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


또 BLEU 점수를 계산하기위해 `sacrebleu`를 설치합니다.

In [5]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.5 MB/s eta 0:00:00


## Dataset

In [6]:
# 데이터 셋을 다운받을 함수를 임포트 합니다.
from datasets import load_dataset

In [7]:
# 데이터을 다운로드 합니다.
dataset = load_dataset("Moo/korean-parallel-corpora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 96215
    })
    validation: Dataset({
        features: ['ko', 'en'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 2000
    })
})

In [8]:
# 데이터 셋을 pandas로 바꿔주기 위해 pandas를 임포드 합니다.
import pandas as pd

In [9]:
# 학습을 위해 활용할 데이터 프레임을 만들어 줍니다.
# 이때, 빠르게 결과값을 보기 위해 학습 데이터는 1000개, 검증, 평가 데이터는 100개의 데이터만 학습에 활용합니다.
# (성능이 더 좋은 모델을 만들려면 잘 정제된 더 많은 데이터를 활용하면 됩니다.)
train = pd.DataFrame()

train['ko'] = dataset['train']['ko']
train['en'] = dataset['train']['en']

In [10]:
valid = pd.DataFrame()

valid['ko'] = dataset['validation']['ko']
valid['en'] = dataset['validation']['en']

In [12]:
test = pd.DataFrame()

test['ko'] = dataset['test']['ko']
test['en'] = dataset['test']['en']

In [13]:
# 필요할 때 파일을 읽어 허깅페이스 데이터 셋을 만들어 활용하기 위해 정데한 데이터 셋을 저장합니다.
train.to_csv("train.csv", index=False)
valid.to_csv("valid.csv", index=False)
test.to_csv("test.csv", index=False)

필요할때 이 파일을 읽어 허깅페이스 데이터셋을 만들 수 있습니다.

아래처럼 데이터 셋과 데이터 셋의 위치를 정의한 사전을 `load_dataset`에 넘기면 됩니다.

In [14]:
data_files = {"train": "train.csv", "valid": "valid.csv", "test": "test.csv"}

In [15]:
dataset =  load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

허깅페이스에서 파이토치 기반 구현을 사용하므로 gpu가 있다면 device를 세팅합니다.

In [17]:
import torch

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

미리 학습된 모델의 체크포인트를 세팅합니다. 여기서 사용할 모델은 한국어와 영어에 미리 학습된 KE-T5모델을 사용합니다. T5모델은 트랜스포머의 인코더, 디코더 구조를 모두 사용하는 모델로 번역기를 만들 때 사용할 수 있는 모델이다. 아래처럼 모델 체크 포인트와 T5 모델에 입력될 최대 토큰 길이를 설정합니다.

In [16]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

## Tokenizer

허깅페이스에서 제공하는 필요 클래스를 임포트 합니다.

먼저 사전학습된 모델을 사용하기 위한 클래스를 임포트 합니다. `AutoTokenizer`는 사전학습된 모델이 사용한 토크나이저를 읽기 위해 필요합니다.

In [19]:
from transformers import AutoTokenizer

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [21]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [22]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서 에러남
                                 remove_columns=dataset["train"].column_names
                                 )

Map:   0%|          | 0/96215 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Model

`AutoModelForSeq2SeqLM`은 시퀀스 투 스퀀스 방식으로 작동하는 사전학습된 모델을 불러 올 때 마지막에 분류기 헤드를 붙여서 모델을 로딩하기 위해 사용합니다.

In [24]:
from transformers import AutoModelForSeq2SeqLM

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

## DataCollator

파이토치에서 모델을 학습시키기 위해서 `DataLoader`를 사용하게 되는데 이 데이터 로더의 역할은 for 루프를 돌면서 데이터 셋으로 부터 샘플을 배치 수만큼 가져오는 것입니다. 이때 샘플을 배치 수만큼 무작위로 가져와 어떤 식으로든 각 샘플을 짝맞춤해서 반환해야하는데 서로 크기가 다른 샘플들을 다루는 경우는 길이를 맞춘다든지 패딩을 한다든지 하는 추가 작업이 필요하게 됩니다. 이런 작업이 일어나는 곳이 `collate_fn`으로 지정되는 함수입니다.

시퀀스 투 시퀀스 모델을 학습시킬때 이런 콜레이터 함수가 하는 전형적인 역할은 입력 또는 출력 문자열을 패딩하고 조금 전 모델에서 알아봤듯이 디코더 타겟을 오른쪽으로 한칸 쉬프트 시켜서 디코더 입력으로 만드는 것 입니다. 이런 작업을 자동으로 처리해주는 클래스가 `DataCollatorForSeq2Seq`입니다.

우선 콜레이터를 만들기 위해서는 토크나이저와 모델을 넘겨야 합니다.

In [26]:
from transformers import DataCollatorForSeq2Seq

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Evaluate

마지막으로 학습한 모델을 측정할 매트릭을 준비해야 합니다.

In [30]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-30-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


아래 함수는 모델의 예측과 레이블을 가지고 bleu를 계산하는 헬퍼 함수로 [트랜스포머 학습 코스 번역기 매트릭](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#metrics)에서 제공하는 코드를 그대로 복사 하면 됩니다.

In [32]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

## Trainer

학습을 간단히 하기위해 허깅페이스에서 제공하는 `Seq2SeqTrainer`클래스를 사용합니다. 학습 세부 조건은 `Seq2SeqTrainingArguments`를 사용하여 설정합니다. 다음 코드로 학습에 필요한 세부 사항을 설정할 수 있습니다.

그리고 학습에 필요한 클래스를 임포트 합니다. 학습에 필요한 설정을 `Seq2SeqTrainingArguments`에 정의하고 실제 학습은 `Seq2SeqTrainer`로 하게 됩니다.

In [28]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [29]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none"
)

세팅을 해서 `training_args`를 만들고 `trainer`를 생성합니다. 지금까지 준비한 `model`, `training_args`, `tokenized_datasets`, `data_collator`, `tokenizer`, `compute_metrics`를 넘기면 됩니다.

In [33]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# 아래 소스코드를 실행시켜 학습을 진행하면 됩니다.
trainer.train()

학습이 끝나면 실험 결과를 저장합니다.

In [ ]:
trainer.save_model("./results")

## Test

학습과 저장을 성공적으로 마쳤으면 다음 명령으로 모델을 불러올 수 있습니다.

In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

로딩된 모델을 테스트하기 위해 다음 두 문장을 준비합니다.

In [ ]:
input_text = [
    "Because deep learning frameworks are well developed, in these days, machine translation system can be built without anyone's help.",
    "This system was made by using HuggingFace's T5 model for a one day"
]

모델이 입력하기위해 토크나이저로 토큰화 시킵니다.

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


`inputs`를 확인해보면 `input_ids`와 `attention_mask`로 토큰화 된것을 알 수 있다. 첫번째 문장이 더 길기 때문에 두번째 문장의 마스크는 마지막에 0으로 패딩된 것도 확인할 수 있습니다.

In [ ]:
inputs

{'input_ids': tensor([[ 8127,  5859,  5789, 22309,     8,    69,   484,  6560,     4,    20,
           572,  1258,     4,  9872, 46301,  1076,   147,    67,  3807,  1215,
          3993,    17,     8,   787,     3,     1],
        [  465,  1076,    62,   565,    81,  1676,   992, 60049,  1044, 17400,
            17,     8,   745,   466,  3900,    40,    16,   165,   688,     1,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0]])}

`model.generate()`에 입력을 넣고 출력을 생성한다. 이때 여러 가능한 결과 중에서 '가장 좋은' 결과를 찾아내기 위해 `num_beams=5`로 설정한다.

In [ ]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

koreans.shape

torch.Size([2, 64])

생성된 결과를 디코딩해보면 다음처럼 번역되는 것을 확인할 수 있습니다.

In [ ]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]